In [94]:
import numpy as np
import pandas as pd
import re
import json

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from collections import Counter

# For cralwer
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et
import requests

import xml.etree.ElementTree as et
import xmltodict

# For visualization
from geopy.geocoders import Nominatim
import folium

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16740\4019966501.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
def query_df_out(query_df,query,version=None):
    a = query_df[query_df["구분"]==query]
    if version =="all":
        b = a
    elif version == 1:
        b = a[query_df["항목구분"]==0]
    else:
        b = a[query_df["항목구분"]==1]
    return b

In [4]:
df_in = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="all")
df_out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="out")
df_item = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="items")

In [5]:
df_in2 = df_in[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
df_out2 = df_out[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)

In [6]:
cate_ = {}
for row,col in df_in2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

for row,col in df_out2.iterrows():
    key_ = col["항목명(영문)"].replace("\\","")
    value_  =col["항목명(국문)"].replace("\\","")
    cate_[key_] = value_

In [7]:
url_main = "http://apis.data.go.kr/B460014/foodBankInfoService2"

In [8]:
encode = "%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D"
decode = "/xc7f1IdclYG5tj6DIg1cU51nehX/zyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA=="

In [9]:
query_1 = {
    "getUserInfo": "이용자", "getFcltyGrpInfo" : "이용시설단체", "getCntrbtrInfo":"기부자",
    "getRceptStat": "접수현황", "getProvdStat": "제공현황",
    "getCnttgInfo":"물품현황","getSpctrInfo":"센터통계", "getPreferInfo":"선호물품"
}

In [10]:
query_1_2 = {
    value:key for key,value in query_1.items()
}

# Censter Listup

In [12]:
df_center = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\getSpctrInfo.xlsx")
df_center.head()

,Unnamed: 0,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소,주소,지원센터주소2,지원센터주소3
0,0,201701,중앙,전국,푸드뱅크,전국푸드뱅크,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,서울특별시 마포구 만리재로 14 한국사회복지회관 4층,서울특별시 마포구 만리재로 14,서울특별시 마포구 만리재로 14
1,1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7,서울 도봉구 창동 1-7,서울 도봉구 창동,서울 도봉구 창동
2,2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7,7,서울특별시 영등포구 도림로61길 7 7,서울특별시 영등포구 도림로61길 7,서울특별시 영등포구 도림로 61길 7
3,3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩,1층,서울특별시 성동구 왕십리로28길 10 준희빌딩 1층,서울특별시 성동구 왕십리로28길 10,서울특별시 성동구 왕십리로 28길 10 준희빌딩
4,4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,서울 종로구 숭인1동 55-2 숭인1동주민센터 4층,서울 종로구 숭인1동,서울 종로구 숭인 1동


In [99]:
target_col = ["지원센터코드","주소","지역코드","지원센터단위코드","지원센터구분코드"]
lambda_ = lambda x: False if x =="푸드뱅크" or x=="푸드마켓" else (True)
df_list_1 = df_center[df_center["지원센터코드"].map(lambda_)]
df_list_1 = df_list_1[target_col].groupby(by=target_col).count()#.unique()
df_list_1 = df_list_1.reset_index(level=[0]).reset_index()
df_list_1

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드
0,전라남도 신안군 압해읍 구항길 92-58,전남,기초,푸드뱅크,1004섬신안기초푸드뱅크
1,경기도 가평군 가평읍 달전로 44 1층,경기,기초,푸드뱅크,가평군푸드뱅크
2,경기도 가평군 가평읍 달전로 44 1층(달전리),경기,기초,푸드뱅크,가평군푸드뱅크
3,서울특별시 강남구 개포로 605 강남 장애인 문화복지관 강남구기초푸드뱅크,서울,기초,푸드뱅크,강남구기초푸드뱅크
4,서울특별시 강남구 학동로 426 (삼성동) 강남구청내 강남사랑나눔이동푸드마켓,서울,기초,푸드마켓,강남이동푸드마켓
...,...,...,...,...,...
684,강원도 화천군 하남면 춘화로 3341 2층 화천군사회복지협의회,강원,기초,푸드뱅크,화천푸드뱅크
685,강원도 횡성군 횡성읍 어수로 46,강원,기초,푸드뱅크,횡성군기초푸드뱅크
686,경상북도 포항시 북구 흥해읍 한동로 43,경북,기초,푸드뱅크,흥해제일기초푸드뱅크
687,광주광역시 남구 진다리로31 13 (백운동),광주,기초,푸드뱅크,희망기초푸드뱅크


In [88]:
def query(post):
    out_ = df_list_1[df_list_1["주소"]==post].reset_index()#
    if len(out_) != 1:
        try:
            result_ = ["multi",out_["지원센터코드"][0]]
        except:
            result_ = ["error",out_["지원센터코드"]]
    else:
        result_ = ["pass",out_["지원센터코드"][0]]
    return result_

In [100]:
(len(df_center))

26453

In [101]:
len(df_list_2)

26453

In [105]:
#lambda_2 = lambda x: True if x=="푸드뱅크" or "푸드마켓" else (False)
df_list_2 = df_center.copy()
list_ = df_list_2["주소"].map(lambda x: query(x))

In [106]:
df_list_2["지원센터코드2"] = [i[1] for i in list_]
df_list_2["estimate"] = [i[0] for i in list_]

In [103]:
Counter(df_list_2[df_list_2["estimate"]=="error"]["주소"])

Counter({'전라남도 순천시 중앙1길 30-18 (장천동)': 19,
         '경기도 화성시 봉담읍 동화길 174 ': 11,
         '대전광역시 대덕구 석봉로57 31 (석봉동)': 6,
         '울산광역시 중구 중앙길 199 (학산동)': 17,
         '강원도 원주시 일산로82 36 (원동)': 15,
         '강원도 원주시 문막읍 동화택지길 9 ': 13,
         '부산광역시 동래구 낙민로 25 (낙민동)부산사회복지종합센터 302호': 11,
         '부산광역시 서구 대영로85 42 (동대신동2가)': 22,
         '부산광역시 북구 덕천로 74 덕천종합사회복지관': 11,
         '인천광역시 서구 봉수대로 806 (연희동)': 11,
         '광주광역시 북구 군왕로207 6 (각화동)': 11,
         '광주광역시 광산구 우산로 17 (우산동, 우산빛여울채)': 11,
         '울산광역시 남구 산업로325 34 (선암동)': 11,
         '경기도 평택시 원평로28 34 (군문동)': 22,
         '경기도 오산시 오산로272 23 2층(오산동)': 11,
         '강원도 강릉시 구정면 구정중앙로 158-10 ': 11,
         '충청북도 옥천군 옥천읍 삼금로4길 6 1층': 11,
         '충청남도 아산시 염치읍 송곡안길 2-23 1층': 11,
         '충청남도 태안군 태안읍 시장5길 20 1층': 11,
         '전라남도 광양시 광양읍 인덕로 977 2층 1호': 11,
         '전라남도 영광군 영광읍 천년로 1351-14 ': 11,
         '경상북도 상주시 냉림1길 43-18 (냉림동) 상주시종합사회복지관': 11,
         '경상남도 김해시 칠산로 128 (신문동) 경상남도광역기부식품등지원센터': 11,
         '경상남도 밀양시 중앙로

# Only considering "경기"

In [110]:
a_ = df_list_2[df_list_2["지역코드"]=="경기"]
Counter(a_[a_["estimate"]=="error"]["주소"])

Counter({'경기도 화성시 봉담읍 동화길 174 ': 11,
         '경기도 평택시 원평로28 34 (군문동)': 22,
         '경기도 오산시 오산로272 23 2층(오산동)': 11,
         '경기도 김포시 김포대로 1521 (장기동)': 11,
         '경기도 평택시 탄현1로 160 , B102호(신장동)': 11,
         '경기도 평택시 탄현1로 160 (신장동) B102호': 11,
         '경기도 안양시 만안구 관악대로 5 지하층 (안양동)': 11,
         '경기도 이천시 경충대로 2602 (중리동)': 11,
         '경기도 안산시 단원구 신촌3길 28 (초지동)': 11,
         '경기도 성남시 수정구 위례순환로 17 (창곡동, 위례31단지) 3109동 에듀센터 1층': 11,
         '경기도 구리시 검배로 154 우남상가 102호 (토평동, 우남아파트)': 11,
         '경기도 안산시 상록구 삼태기1길 37 1층 (이동)': 11,
         '경기도 화성시 봉담읍 동화길 174 101호': 7})

In [114]:
df_list_1[df_list_1["지역코드"]=="경기"].to_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\경기\center_list.xlsx")

# Extract lon,lat only 경기

In [120]:
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [115]:
df_target_1 = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\경기\center_list_after.xlsx")

In [119]:
df_target_1.head()

,Unnamed: 0,주소,주소_after,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,estimate
0,1,경기도 가평군 가평읍 달전로 44 1층,경기도 가평군 가평읍 달전로 44,경기,기초,푸드뱅크,가평군푸드뱅크,NaN
1,27,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기도 화성시 봉담읍 덕우공단2길 48,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고_1,NaN
2,28,경기도 화성시 정남면 괘랑1길 42-30 A동,경기도 화성시 정남면 괘랑1길 42-30,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고_2,NaN
3,29,경기도 양주시 칠봉산로 303 (봉양동),경기도 양주시 칠봉산로 303,경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고_3,NaN
4,30,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기도 수원시 장안구 서부로 2139,경기,광역,푸드뱅크,경기도광역푸드뱅크_1,NaN


In [125]:
tq = tqdm(range(len(df_target_1)))
lat_lng_list = []
for i in tq:
    lat_lng_list.append(geocoding(df_target_1["주소_after"][i]))

  0%|          | 0/140 [00:00<?, ?it/s]

In [126]:
df_target_1["lat_lng"] = lat_lng_list
df_target_1["lat"] = [i[0] for i in lat_lng_list]
df_target_1["lng"] = [i[1] for i in lat_lng_list]

In [127]:
df_target_1.head()

,Unnamed: 0,주소,주소_after,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,estimate,lat_lng,lat,lng
0,1,경기도 가평군 가평읍 달전로 44 1층,경기도 가평군 가평읍 달전로 44,경기,기초,푸드뱅크,가평군푸드뱅크,NaN,"[37.8152203, 127.517003]",37.815220,127.517003
1,27,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기도 화성시 봉담읍 덕우공단2길 48,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고_1,NaN,"[0, 0]",0.000000,0.000000
2,28,경기도 화성시 정남면 괘랑1길 42-30 A동,경기도 화성시 정남면 괘랑1길 42-30,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고_2,NaN,"[0, 0]",0.000000,0.000000
3,29,경기도 양주시 칠봉산로 303 (봉양동),경기도 양주시 칠봉산로 303,경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고_3,NaN,"[37.8653252, 127.0642794]",37.865325,127.064279
4,30,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기도 수원시 장안구 서부로 2139,경기,광역,푸드뱅크,경기도광역푸드뱅크_1,NaN,"[37.3179173, 126.9852447]",37.317917,126.985245


In [129]:
for i in df_target_1[df_target_1["lat"]==0]["주소_after"]:
    print(i)

경기도 화성시 봉담읍 덕우공단2길 48
경기도 화성시 정남면 괘랑1길 42-30
경기도 광명시 소하로 88
경기도 광주시 고불로 40-11
경기도 군포시 오금로 102
경기도 안양시 만안구 장내로 75 42
경기도 용인시 처인구 이동면 백옥대로 624 2-7 
경기도 김포시 북변중로 65 4-8
경기도 김포시 하성면 봉성로 159 71-5
경기도 양평군 청운면 백동길 1-3
경기 남양주시 화도읍 마석우리 79
경기도 남양주시 화도읍 경춘로달뫼1길 20-8
경기도 광주시 곤지암읍 곤지암로11 8
경기 동두천시 광암동 518-1
경기도 동두천시 광암로6 2
경기도 화성시 병점중앙로10 10
경기도 부천시 원미로 235
경기도 부천시 신흥로275 19
경기도 부천시 신흥로275 19
경기도 화성시 서신면 수풀오얏길 65-11
경기도 시흥시 봉우재로51 3
경기도 이천시 경충대로2638 35
경기 안성시 공도읍 진사리 진사길 79-17
경기도 평택시 안중읍 안중로8 23-11
경기도 성남시 중원구 성남대로1151 20-7
경기도 성남시 분당구 미금로 246
경기도 성남시 분당구 탄천로 215
경기도 이천시 마장면 작촌로 24-23 
경기도 포천시 소흘읍 송우로132 56
경기 수원시 권선구 탑동 869-9
경기도 수원시 권선구 효탑로 24-17
경기도 수원시 장안구 장안로63 8
경기도 광명시 광삼로 9
경기도 광명시 금하로527 12
경기도 시흥시 오동마을로45 7
경기도 시흥시 신천로79 26
경기도 용인시 처인구 금령로71 8
경기도 안산시 상록구 충장로6안길 16
경기도 안산시 단원구 화정로안길 53
경기도 안산시 상록구 원당골3길 6-1
경기도 안산시 단원구 예술1길 6
경기도 안산시 단원구 예술대학로5길 33
경기도 안성시 죽산면 죽주로 292-22
경기도 안성시 중앙로451 11
경기도 안성시 일죽면 금일로 93
경기 안양시 만안구 안양2동 688-54
경기도 안양시 만안구 안양로360 26
경기도 안양시 만안구 장내로 113
경기 양주시 광적면 가납리 

In [140]:
geocoding("경기도 화성시 봉담읍 덕우리")

[0, 0]

# 아래의 사이트로 그냥 cralwer를 만드는게 빠를듯하다

In [141]:
"https://address.dawul.co.kr/"

'https://address.dawul.co.kr/'